In [153]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
import pandas as pd
from sklearn.model_selection import train_test_split

In [187]:
# uses the file tanmay made for now, need to upadte to use script to generate filtered dataset
df_filtered = pd.read_csv('data/processed/filtered_data.csv')

In [188]:
df_filtered.describe(include='all')

,Unnamed: 0,MainBranch,Employment,RemoteWork,EdLevel,YearsCode,YearsCodePro,DevType,OrgSize,Country,...,ToolsTechHaveWorkedWith,NEWCollabToolsHaveWorkedWith,OpSysProfessional use,VersionControlSystem,VCInteraction,OfficeStackAsyncHaveWorkedWith,Age,WorkExp,ICorPM,ConvertedCompYearly
count,8042.000000,8042,8042,8042,8042,8042,8042,8042,8042,8042,...,6444,7998,7792,8042,7950,5959,8042,5610.000000,5558,8.042000e+03
unique,NaN,2,9,3,9,52,51,2110,10,2,...,441,2422,40,16,15,398,8,NaN,2,NaN
top,NaN,I am a developer by profession,"Employed, full-time",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,5,"Developer, full-stack","10,000 or more employees",United States of America,...,Docker,Visual Studio Code,macOS,Git,Command-line,Confluence;Jira Work Management,25-34 years old,NaN,Independent contributor,NaN
freq,NaN,7340,7239,4960,4601,612,560,1217,1589,6874,...,592,777,2287,7119,2177,1256,3413,NaN,4799,NaN
mean,36128.701069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.129768,NaN,3.526406e+05
std,21238.664698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.026063,NaN,1.169285e+06
min,11.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,3.600000e+01
25%,15715.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,9.760500e+04
50%,39136.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,1.380000e+05
75%,53352.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.000000,NaN,1.951575e+05


In [189]:
df_filtered = df_filtered.query('ConvertedCompYearly < 500000')

In [191]:
df_filtered.describe(include='all')

,Unnamed: 0,MainBranch,Employment,RemoteWork,EdLevel,YearsCode,YearsCodePro,DevType,OrgSize,Country,...,ToolsTechHaveWorkedWith,NEWCollabToolsHaveWorkedWith,OpSysProfessional use,VersionControlSystem,VCInteraction,OfficeStackAsyncHaveWorkedWith,Age,WorkExp,ICorPM,ConvertedCompYearly
count,7504.000000,7504,7504,7504,7504,7504,7504,7504,7504,7504,...,6007,7463,7276,7504,7416,5568,7504,5266.000000,5219,7504.000000
unique,NaN,2,9,3,9,52,51,1970,10,2,...,428,2301,40,16,15,387,8,NaN,2,NaN
top,NaN,I am a developer by profession,"Employed, full-time",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,5,"Developer, full-stack",100 to 499 employees,United States of America,...,Docker,Visual Studio Code,macOS,Git,Command-line,Confluence;Jira Work Management,25-34 years old,NaN,Independent contributor,NaN
freq,NaN,6853,6772,4650,4292,569,517,1142,1487,6402,...,556,724,2145,6650,2019,1192,3186,NaN,4514,NaN
mean,36253.397388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.151348,NaN,144257.003065
std,21240.399916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.037086,NaN,72781.842552
min,11.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,36.000000
25%,15727.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,94482.000000
50%,39222.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,130000.000000
75%,53495.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.000000,NaN,178000.000000


In [190]:
train_df_filtered, test_df_filtered = train_test_split(df_filtered, test_size=0.10, random_state=123)

train_df_filtered.describe(include='all')

,Unnamed: 0,MainBranch,Employment,RemoteWork,EdLevel,YearsCode,YearsCodePro,DevType,OrgSize,Country,...,ToolsTechHaveWorkedWith,NEWCollabToolsHaveWorkedWith,OpSysProfessional use,VersionControlSystem,VCInteraction,OfficeStackAsyncHaveWorkedWith,Age,WorkExp,ICorPM,ConvertedCompYearly
count,6753.000000,6753,6753,6753,6753,6753,6753,6753,6753,6753,...,5406,6716,6547,6753,6673,5017,6753,4731.000000,4689,6753.000000
unique,NaN,2,9,3,9,52,51,1807,10,2,...,409,2114,38,15,15,373,8,NaN,2,NaN
top,NaN,I am a developer by profession,"Employed, full-time",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,5,"Developer, full-stack",100 to 499 employees,United States of America,...,Docker,Visual Studio Code,macOS,Git,Command-line,Confluence;Jira Work Management,25-34 years old,NaN,Independent contributor,NaN
freq,NaN,6161,6086,4181,3851,516,475,1013,1343,5756,...,514,645,1917,5981,1823,1071,2895,NaN,4044,NaN
mean,36264.240338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.147749,NaN,143988.600622
std,21212.800625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.081693,NaN,72454.997649
min,11.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,36.000000
25%,15729.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,94500.000000
50%,39243.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,130000.000000
75%,53463.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.000000,NaN,177500.000000


In [157]:
# def multianswer_col_trans(df, col_name):
    
#     cv = CountVectorizer(tokenizer=lambda text: text.split(';'))
#     df[col_name] = df[col_name].fillna('unspecified')
#     sr_col_name = df[col_name]
#     cv.fit(sr_col_name)
#     columns = cv.get_feature_names_out()
#     df_encoded = pd.DataFrame(cv.transform(sr_col_name).toarray(),
#                         columns= columns,
#                         index = sr_col_name.index)
#     df_encoded = df_encoded.add_prefix(col_name + "_")
#     return df_encoded

In [158]:
def write_na_values_for_cols(df, list_of_cols, fillna_base_text = 'unspecified'):
    temp_df = df.copy()
    for col in list_of_cols:
        fillna_text_final = col + "_" + fillna_base_text
        temp_df[col] = temp_df[col].fillna(fillna_text_final)
        
    return temp_df

In [159]:
# Returns float values for different string inputs
def convert2float(x):
    if  x == 'More than 50 years' :
        return float(50)
    elif x == 'Less than 1 year':
        return float(0)
    else:
        return float(x)

In [160]:
# converts string year values to float
train_df_filtered['YearsCode'] = train_df_filtered['YearsCode'].apply(lambda x: convert2float(x))
train_df_filtered['YearsCodePro'] = train_df_filtered['YearsCodePro'].apply(lambda x: convert2float(x))
test_df_filtered['YearsCode'] = test_df_filtered['YearsCode'].apply(lambda x: convert2float(x))
test_df_filtered['YearsCodePro'] = test_df_filtered['YearsCodePro'].apply(lambda x: convert2float(x))

In [161]:
# order for ordinal columns
education_order = ['Something else', 'Primary/elementary school', 'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
                   'Some college/university study without earning a degree', 'Associate degree (A.A., A.S., etc.)', "Bachelor’s degree (B.A., B.S., B.Eng., etc.)", 
                   "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",'Professional degree (JD, MD, etc.)', 'Other doctoral degree (Ph.D., Ed.D., etc.)']

age_order = ['Prefer not to say', 'Under 18 years old', '18-24 years old','25-34 years old','35-44 years old','45-54 years old', '55-64 years old','65 years or older']

In [162]:
# columns for preprocessing 
multianswer_cols = [
'DevType',
'LanguageHaveWorkedWith',
'DatabaseHaveWorkedWith',
'PlatformHaveWorkedWith',
'WebframeHaveWorkedWith',
'MiscTechHaveWorkedWith',
'ToolsTechHaveWorkedWith',
'NEWCollabToolsHaveWorkedWith',
'OpSysProfessional use',
'VCInteraction',
'VersionControlSystem',
'OfficeStackAsyncHaveWorkedWith',
'Employment']

numeric_cols = ['YearsCode', 'YearsCodePro', 'WorkExp']

ordinal_edu = ['EdLevel']

ordinal_age = ['Age']

binary_cols = ['MainBranch', 'Country']

categorical_cols = ['OrgSize', 'RemoteWork']

# passthrough_cols = []

drop_cols = ['ICorPM', 'Unnamed: 0']

In [163]:
# replace nan values from the multianswer_cols
train_df_filtered = write_na_values_for_cols(train_df_filtered, multianswer_cols)

In [164]:
train_df_filtered.describe(include='all')

,Unnamed: 0,MainBranch,Employment,RemoteWork,EdLevel,YearsCode,YearsCodePro,DevType,OrgSize,Country,...,ToolsTechHaveWorkedWith,NEWCollabToolsHaveWorkedWith,OpSysProfessional use,VersionControlSystem,VCInteraction,OfficeStackAsyncHaveWorkedWith,Age,WorkExp,ICorPM,ConvertedCompYearly
count,6753.000000,6753,6753,6753,6753,6753.000000,6753.000000,6753,6753,6753,...,6753,6753,6753,6753,6753,6753,6753,4731.000000,4689,6753.000000
unique,NaN,2,9,3,9,NaN,NaN,1807,10,2,...,410,2115,39,15,16,374,8,NaN,2,NaN
top,NaN,I am a developer by profession,"Employed, full-time",Fully remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",NaN,NaN,"Developer, full-stack",100 to 499 employees,United States of America,...,ToolsTechHaveWorkedWith_unspecified,Visual Studio Code,macOS,Git,Command-line,OfficeStackAsyncHaveWorkedWith_unspecified,25-34 years old,NaN,Independent contributor,NaN
freq,NaN,6161,6086,4181,3851,NaN,NaN,1013,1343,5756,...,1347,645,1917,5981,1823,1736,2895,NaN,4044,NaN
mean,36264.240338,NaN,NaN,NaN,NaN,16.688139,11.546720,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.147749,NaN,143988.600622
std,21212.800625,NaN,NaN,NaN,NaN,10.659747,9.476051,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.081693,NaN,72454.997649
min,11.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,36.000000
25%,15729.000000,NaN,NaN,NaN,NaN,8.000000,5.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,94500.000000
50%,39243.000000,NaN,NaN,NaN,NaN,14.000000,9.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,130000.000000
75%,53463.000000,NaN,NaN,NaN,NaN,23.000000,16.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.000000,NaN,177500.000000


In [166]:
numeric_transformer = make_pipeline(SimpleImputer(strategy='most_frequent'), StandardScaler())

ordinal_edu_transformer = make_pipeline(OrdinalEncoder(categories=[education_order], dtype=int))

ordinal_age_transformer = make_pipeline(OrdinalEncoder(categories=[age_order], dtype=int))

binary_transformer = make_pipeline(OneHotEncoder(drop='if_binary', dtype=int))

categorical_transformer = make_pipeline(OneHotEncoder(handle_unknown='ignore', sparse=False))


preprocessor = make_column_transformer(
               (numeric_transformer, numeric_cols),
               (ordinal_edu_transformer, ordinal_edu),
               (ordinal_age_transformer, ordinal_age),
               (binary_transformer, binary_cols),
               (categorical_transformer, categorical_cols),
               # ('passthrough', passthrough_cols),
               ('drop', drop_cols),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[0]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[1]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[2]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[3]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[4]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[5]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[6]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[7]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[8]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[9]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[10]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[11]),
               (CountVectorizer(tokenizer=lambda text: text.split(';')), multianswer_cols[12])
)

In [167]:
# view preprocessor
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['YearsCode', 'YearsCodePro', 'WorkExp']),
                                ('pipeline-2',
                                 Pipeline(steps=[('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Something '
                                                                              'else',
                                                                              'Primary/elementary '
                                                                              'school',
                                                                              'Secondary '
                                                                              'school '
                                                                              '(e.g. '
                                                                              'Am...
                                 'VCInteraction'),
                                ('countvectorizer-11',
                                 CountVectorizer(tokenizer=<function <lambda> at 0x00000228DBC352D0>),
                                 'VersionControlSystem'),
                                ('countvectorizer-12',
                                 CountVectorizer(tokenizer=<function <lambda> at 0x00000228DBC36170>),
                                 'OfficeStackAsyncHaveWorkedWith'),
                                ('countvectorizer-13',
                                 CountVectorizer(tokenizer=<function <lambda> at 0x00000228DBC35A20>),
                                 'Employment')])

In [168]:
X_train = train_df_filtered.drop(columns=["ConvertedCompYearly"])
y_train = train_df_filtered["ConvertedCompYearly"]

X_test = test_df_filtered.drop(columns=["ConvertedCompYearly"])
y_test = test_df_filtered["ConvertedCompYearly"]

In [169]:
# fit preprocessor
preprocessor.fit_transform(X_train)

# columns that dont contain multiple answers per response
column_names = (numeric_cols + ordinal_edu + ordinal_age + binary_cols + categorical_cols + drop_cols + passthrough_cols)

# creates list of new column names from preprocessing pipelines
transformed_column_names = []

for i in range(1,6):
    temp_names = preprocessor.named_transformers_['pipeline-'+str(i)].get_feature_names_out().tolist()
    transformed_column_names += temp_names

for i in range(1,14):
    temp_names = preprocessor.named_transformers_['countvectorizer-'+str(i)].get_feature_names_out().tolist()
    # print(temp_names)
    for name in temp_names:
        name = multianswer_cols[i-1] + "_" + name
        transformed_column_names.append(name)

In [170]:
print(len(transformed_column_names))
# transformed_column_names

259


In [171]:
# # fit and transform the columns that dont contain multiple answers per response

# transformed_train_df = pd.DataFrame(
#                        preprocessor.fit_transform(non_multianswer_train_df),
#                        columns = transformed_column_names)

# transformed_train_df

In [172]:
# # creates final transformed train df

# for column in multianswer_cols:
#     # count vectorizer for multi answer columns
#     temp_col = multianswer_col_trans(train_df_filtered, column)
#     temp_col.reset_index(drop=True, inplace=True)
#     transformed_train_df = pd.concat([transformed_train_df, temp_col], axis=1)

# transformed_train_df


# Feature Selection

In [173]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

In [174]:
X_train_enc = pd.DataFrame(
    data=preprocessor.transform(X_train).todense(), 
    index=X_train.index, 
    columns=transformed_column_names
)
# X_train_enc

In [175]:
select_lr = SelectFromModel(Ridge(), threshold="0.8*mean")

pipe_rf_model_based = make_pipeline(
    preprocessor, select_lr, RandomForestRegressor(random_state=16)
)
pipe_rf_model_based.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['YearsCode', 'YearsCodePro',
                                                   'WorkExp']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('ordinalencoder',
                                                                   OrdinalEncoder(categories=[['Something '
                                                                                               'else',
                                                                                               'Primary/elementary '
                                                                                               '...
                                                 ('countvectorizer-12',
                                                  CountVectorizer(tokenizer=<function <lambda> at 0x00000228DBC36170>),
                                                  'OfficeStackAsyncHaveWorkedWith'),
                                                 ('countvectorizer-13',
                                                  CountVectorizer(tokenizer=<function <lambda> at 0x00000228DBC35A20>),
                                                  'Employment')])),
                ('selectfrommodel',
                 SelectFromModel(estimator=Ridge(), threshold='0.8*mean')),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=16))])

In [176]:
model_based_mask = pipe_rf_model_based.named_steps["selectfrommodel"].get_support()
mb_selected_feats = X_train_enc.columns[model_based_mask]
fs = mb_selected_feats.tolist()

In [177]:
# fs

# Model Selection

In [178]:
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score, cross_validate

In [179]:
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [180]:
cross_val_results_reg = {}
cross_val_results_reg_fs = {}

models = {
    "Baseline": DummyRegressor(),
    "KNN Regressor": KNeighborsRegressor(),
    "Ridge": Ridge(),
    "Random Forest Regressor": RandomForestRegressor(),
    "Lasso": Lasso(),
}

score_types_reg = {
    #"neg_mean_squared_error": "neg_mean_squared_error",
    #"neg_root_mean_squared_error": "neg_root_mean_squared_error",
    "neg_mape": "neg_mean_absolute_percentage_error", 
    "r2": "r2",
}

In [181]:
def corss_validate_result(model_name, model_type, fs):
    pipe = make_pipeline(
        preprocessor,
        model_type
    )
    cross_val_results_reg[model_name] = pd.DataFrame(cross_validate(pipe,
                                                                    X_train,
                                                                    y_train, 
                                                                    return_train_score=True, 
                                                                    scoring=list(score_types_reg.values()))).agg(['mean', 'std']).round(3).T
    
    # cross_val_results_reg_fs[model_name] = pd.DataFrame(cross_validate(pipe,
    #                                                                    X_train[fs],
    #                                                                    y_train, 
    #                                                                    return_train_score=True, 
    #                                                                    scoring=list(score_types_reg.values()))).agg(['mean', 'std']).round(3).T

In [182]:
for model_item in models.items():
    model_name = model_item[0]
    model_type = model_item[1]
    corss_validate_result(model_name, model_type, fs)

In [183]:
pd.concat(
    {key: pd.DataFrame(value) for key, value in cross_val_results_reg.items()}, 
    axis=1
)

Baseline        KNN Regressor         \
                                             mean    std          mean    std   
fit_time                                    1.200  0.327         1.104  0.302   
score_time                                  0.346  0.139         1.535  0.730   
test_neg_mean_absolute_percentage_error    -4.119  2.932        -4.444  3.670   
train_neg_mean_absolute_percentage_error   -4.114  0.724        -3.545  0.769   
test_r2                                    -0.001  0.001         0.170  0.019   
train_r2                                    0.000  0.000         0.442  0.004   

                                          Ridge         \
                                           mean    std   
fit_time                                  1.171  0.131   
score_time                                0.333  0.041   
test_neg_mean_absolute_percentage_error  -4.520  3.820   
train_neg_mean_absolute_percentage_error -4.280  0.846   
test_r2                                   0.303  0.026   
train_r2                                  0.380  0.005   

                                         Random Forest Regressor         \
                                                            mean    std   
fit_time                                                 241.477  9.477   
score_time                                                 0.416  0.097   
test_neg_mean_absolute_percentage_error                   -4.103  3.135   
train_neg_mean_absolute_percentage_error                  -1.546  0.292   
test_r2                                                    0.324  0.012   
train_r2                                                   0.905  0.001   

                                          Lasso         
                                           mean    std  
fit_time                                  1.756  0.229  
score_time                                0.313  0.020  
test_neg_mean_absolute_percentage_error  -4.530  3.849  
train_neg_mean_absolute_percentage_error -4.291  0.857  
test_r2                                   0.301  0.028  
train_r2                                  0.381  0.005

In [184]:
# pd.concat(
#     {key: pd.DataFrame(value) for key, value in cross_val_results_reg_fs.items()}, 
#     axis=1
# )

In [185]:
mean_std_cross_val_scores(
    pipe_rf_model_based, X_train, y_train, return_train_score=True
)

fit_time       65.419 (+/- 6.762)
score_time      0.419 (+/- 0.140)
test_score      0.275 (+/- 0.025)
train_score     0.899 (+/- 0.002)
dtype: object